In [74]:
using DataFrames, LinearAlgebra, Distributions, Revise, Serialization, DebuggingUtilities

In [75]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [76]:
# Get data
df = deserialize("jls/df.jls");
dropmissing!(df);

---
#### Make economy and parameters for BLP

In [77]:
firm_IDs_long = df.j;
tract_IDs_long = df.t;
# X = Matrix(df[!, [:x1, :x2, :z1, :z2]])
X = Matrix(df[!, [:x1, :x2]])
D = Matrix(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 100;

In [78]:
σ = ones(size(D)[2]);
pars = m.set_Pars(K = 2, nI = nI, δs=ones(nJ,1));
serialize("jls/pars.jls", pars)

In [79]:
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );

serialize("jls/ec.jls", ec)
# serialize("/export/storage_adgandhi/MiscLi/factract/analysis/ec.jls", ec)

Economy with 394 firms and 3110 tracts.


---

### Make economy for NLLS

In [84]:
firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix(df[!, [:x1, :x2]])
D = Matrix(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 1;

ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );

serialize("jls/ec_nlls.jls", ec)

Economy with 394 firms and 3110 tracts.


---

## Computation

In [81]:
Threads.nthreads()

32

In [82]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
deltas, q_iter = m.compute_deltas(ec, pars, σ,max_iter=10000, verbose=true);
# serialize("jls/deltas.jls", deltas);

dist = 9.667152460757222e-7, iterations = 276


In [13]:
using BenchmarkTools

In [73]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.274327 seconds (519.15 k allocations: 61.624 MiB, 29.17% compilation time)


In [74]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 10
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.076545 seconds (900.14 k allocations: 101.865 MiB)


In [75]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1000
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  1.515471 seconds (22.36 M allocations: 1.873 GiB, 5.73% gc time)


In [17]:
Threads.nthreads()

16

In [18]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`